In [ ]:
import os
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import time
from collections import Counter # Untuk menghitung frekuensi jika diperlukan nanti

# --- Konfigurasi ---
NAMA_FOLDER_INPUT = 'input'
NAMA_FOLDER_OUTPUT_STEMMED_RELATIF = 'output/sastrawi' # Output hasil stemming
NAMA_FOLDER_OUTPUT_CLEANED_RELATIF = 'output/cleaned' # Output hasil cleaning saja
# --- Akhir Konfigurasi ---

# Dapatkan path direktori kerja saat ini
base_dir = os.getcwd()
print(f"Menggunakan direktori kerja saat ini sebagai basis: {base_dir}")
input_folder_path = os.path.join(base_dir, NAMA_FOLDER_INPUT)
output_folder_stemmed_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_STEMMED_RELATIF)
output_folder_cleaned_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_CLEANED_RELATIF) # Path untuk file cleaned

# --- Fungsi untuk Membersihkan Teks ---
def bersihkan_teks(teks):
    teks = teks.lower()
    teks = re.sub(r"\d+", "", teks)
    tanda_baca_escaped = re.escape(string.punctuation)
    teks = re.sub(r'[' + tanda_baca_escaped + ']', '', teks)
    teks = re.sub(r'\s+', ' ', teks).strip()
    return teks
# --- Akhir Fungsi Cleaning ---

# 1. Inisialisasi Stemmer Sastrawi
print("Menginisialisasi stemmer Sastrawi...")
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("Stemmer siap.")

# 2. Buat folder output (cleaned dan stemmed) jika belum ada
for folder_path in [output_folder_stemmed_path, output_folder_cleaned_path]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder output dibuat: {folder_path}")
    else:
        print(f"Folder output sudah ada: {folder_path}")

# --- Variabel untuk Evaluasi Reduksi Kosakata ---
unique_words_cleaned = set()
unique_words_stemmed = set()
total_words_cleaned = 0
total_words_stemmed = 0
# --- Akhir Variabel Evaluasi ---


# 3. Proses setiap file dalam folder input
print(f"\nMemulai proses cleaning dan stemming dari folder: {input_folder_path}")
start_time = time.time()

try:
    if not os.path.isdir(input_folder_path):
        raise FileNotFoundError(f"Folder input '{input_folder_path}' tidak ditemukan atau bukan direktori.")

    list_file = os.listdir(input_folder_path)
    processed_files = 0
    skipped_files = 0

    if not list_file:
        print("Folder input kosong, tidak ada file yang diproses.")

    for filename in list_file:
        input_file_path = os.path.join(input_folder_path, filename)

        if os.path.isfile(input_file_path) and filename.lower().endswith('.txt'):
            output_file_stemmed_path = os.path.join(output_folder_stemmed_path, filename)
            output_file_cleaned_path = os.path.join(output_folder_cleaned_path, filename) # Path file output cleaned

            print(f"  -> Memproses file: {filename}...")

            try:
                # Baca konten file input
                with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f_in:
                    original_text = f_in.read()

                # Lakukan Cleaning Teks
                cleaned_text = bersihkan_teks(original_text)

                # *** Simpan Teks yang Hanya Dibersihkan (Untuk Inspeksi Manual) ***
                with open(output_file_cleaned_path, 'w', encoding='utf-8') as f_clean:
                    f_clean.write(cleaned_text)

                # Lakukan stemming pada teks yang sudah dibersihkan
                stemmed_text = stemmer.stem(cleaned_text)

                # Tulis hasil stemming ke file output
                with open(output_file_stemmed_path, 'w', encoding='utf-8') as f_stemmed:
                    f_stemmed.write(stemmed_text)

                # *** Update data untuk evaluasi reduksi kosakata ***
                cleaned_words = cleaned_text.split() # Split berdasarkan spasi
                stemmed_words = stemmed_text.split()

                unique_words_cleaned.update(cleaned_words)
                unique_words_stemmed.update(stemmed_words)
                total_words_cleaned += len(cleaned_words)
                total_words_stemmed += len(stemmed_words) # Jumlah kata setelah stemming mungkin berubah

                processed_files += 1

            except Exception as e:
                print(f"     ERROR saat memproses file {filename}: {e}")
        else:
            if os.path.isfile(input_file_path):
                 print(f"  -> Melewati file non-txt: {filename}")
            else:
                 print(f"  -> Melewati item yang bukan file: {filename}")
            skipped_files += 1

    end_time = time.time()
    total_time = end_time - start_time

    print(f"\nProses cleaning dan stemming selesai.")
    print(f"Jumlah file .txt yang diproses: {processed_files}")
    if skipped_files > 0:
        print(f"Jumlah item non-txt atau subfolder yang dilewati: {skipped_files}")
    print(f"Hasil stemming disimpan di folder: {output_folder_stemmed_path}")
    print(f"Hasil cleaning (tanpa stemming) disimpan di folder: {output_folder_cleaned_path}") # Info baru
    print(f"Total waktu eksekusi: {total_time:.2f} detik")

    # --- Cetak Hasil Evaluasi Reduksi Kosakata ---
    print("\n--- Evaluasi Reduksi Kosakata ---")
    count_unique_cleaned = len(unique_words_cleaned)
    count_unique_stemmed = len(unique_words_stemmed)
    print(f"Total kata (tokens) setelah cleaning: {total_words_cleaned}")
    print(f"Total kata (tokens) setelah stemming: {total_words_stemmed}") # Mungkin tidak terlalu informatif, tapi bisa jadi indikasi
    print(f"Jumlah kata unik (types) setelah cleaning: {count_unique_cleaned}")
    print(f"Jumlah kata unik (types) setelah stemming: {count_unique_stemmed}")

    if count_unique_cleaned > 0:
        reduction_percentage = ((count_unique_cleaned - count_unique_stemmed) / count_unique_cleaned) * 100
        print(f"Persentase reduksi kosakata unik: {reduction_percentage:.2f}%")
    else:
        print("Tidak ada kata unik yang ditemukan untuk menghitung reduksi.")
    # --- Akhir Cetak Hasil Evaluasi ---


except FileNotFoundError as fnf_error:
    print(f"ERROR: {fnf_error}")
    print("Pastikan nama folder input sudah benar dan folder tersebut ada.")
except Exception as e:
    print(f"Terjadi kesalahan umum yang tidak terduga: {e}")

Menggunakan direktori kerja saat ini sebagai basis: /home/xerces/project/stemming-project
Menginisialisasi stemmer Sastrawi...
Stemmer siap.
Folder output sudah ada: /home/xerces/project/stemming-project/output/sastrawi

Memulai proses stemming dari folder: /home/xerces/project/stemming-project/input
  -> Memproses file: Kierkegaard_ A Brief Introduction_1.txt...

Proses stemming selesai.
Jumlah file .txt yang diproses: 1
Hasil disimpan di folder: /home/xerces/project/stemming-project/output/sastrawi
Total waktu eksekusi: 13.40 detik
